Input libraries


In [1]:
import pandas as pd
import numpy as np

Importing our train dataframe



In [2]:
traindata = pd.read_csv('income.train.csv')   # our full train dataframe

"totalentropy" is a function to calculate the entropy of an entire dataset

In [3]:
def totalentropy(dataset):
    ent = 0
    totalvalues = dataset.shape[0]
    sampleval = dataset[dataset.columns[-1]].value_counts()
    for i in range(len(sampleval.value_counts())):
        ent  = ent  - (sampleval[i]/totalvalues)* np.log2(sampleval[i]/totalvalues)

    return ent

"infogain" is a function to calculate the information gain of each feature in our dataset and store them in a 1-D array

In [4]:
def infogain (dataset, yesval, noval):
    gain = [0]*(len(dataset.columns)-1)
    I = [0]*(len(dataset.columns)-1)
    for c in range (len(dataset.columns)-1):
        H = [0]*len(dataset.groupby(dataset.columns[c]).label.count())
        totalvals = dataset.groupby(dataset.columns[c]).label.count()
        morevals = dataset.groupby([dataset.columns[c],dataset.columns[-1]]).label.count().unstack().groupby(dataset.columns[c])[yesval].sum().astype('Int32')
        lessvals = dataset.groupby([dataset.columns[c],dataset.columns[-1]]).label.count().unstack().groupby(dataset.columns[c])[noval].sum().astype('Int32')
        for i in range (len(dataset.groupby(dataset.columns[c]).label.count())):
            if (morevals[i]==totalvals[i] or lessvals[i]==totalvals[i]):
                H[i] = 0
            else: 
                H[i]  = H[i] - (morevals[i]/totalvals[i])* np.log2(morevals[i]/totalvals[i]) - (lessvals[i]/totalvals[i])* np.log2(lessvals[i]/totalvals[i])
        P = [0]*len(dataset.groupby(dataset.columns[c]).label.count())
        for i in range (len(dataset.groupby(dataset.columns[c]).label.count())):
            P[i] = totalvals[i]/dataset.shape[0]
        
        for j in range (len(dataset.groupby(dataset.columns[c]).label.count())):
            I[c] = I[c] + H[j]*P[j]
            
        gain[c] = totalentropy(dataset) - I[c]
    return gain

"id3" is the function that builds our decision tree using the ID3 algorithm in the form of a python dictionary

In [5]:
def id3(dataset, yesval, noval):
    
    
    # We calculate infogain for any given dataset
    gains = infogain(dataset, yesval, noval)
    
    
        
    # We find the feature with highest gain
    highestgainindex = np.where(gains == np.amax(gains))
    
    #if there only exist one feature with the max gain value we proceed normally
    if (len(highestgainindex[0])==1):
        index = highestgainindex[0][0]
        node = dataset.columns[index]
        
    #if there are more than one features with the max gain value we pick the first feature and eliminate the rest
    else:
       
       
        for i in range (len(highestgainindex[0])-1):
            dataset = dataset.drop(columns=dataset.columns[highestgainindex[0][len(highestgainindex[0])-1-i]])
        if (len(dataset.columns)>2):
            gains = infogain(dataset, yesval, noval)
            highestgainindex = np.where(gains == np.amax(gains))
            index = highestgainindex[0][0]
            node = dataset.columns[index]
            
        #if by eliminating the features with same info gain, our dataset is left with only one feature in total (with the excption of 'label')
        else:
            index = 0
            node = dataset.columns[index]
            

        
    
    
    tree = { node:{}}
    for i in range(len(dataset.groupby(dataset.columns[index]).label.count())):
        totalvals = dataset.groupby(dataset.columns[index]).label.count()
        morevals = dataset.groupby([dataset.columns[index],dataset.columns[-1]]).label.count().unstack().groupby(dataset.columns[index])[yesval].sum().astype('Int32')
        lessvals = dataset.groupby([dataset.columns[index],dataset.columns[-1]]).label.count().unstack().groupby(dataset.columns[index])[noval].sum().astype('Int32')

        if (morevals[i]==totalvals[i]):
            tree[node][dataset.groupby(dataset.columns[index]).label.count().index[i]] = yesval 
        elif (lessvals[i]==totalvals[i]):     
            tree[node][dataset.groupby(dataset.columns[index]).label.count().index[i]] =  noval 

            
        else:
            if ((len(dataset.columns)-1)==1):
                morevals = dataset.groupby([dataset.columns[index],dataset.columns[-1]]).label.count().unstack().groupby(dataset.columns[index])[yesval].sum().astype('Int32')
                lessvals = dataset.groupby([dataset.columns[index],dataset.columns[-1]]).label.count().unstack().groupby(dataset.columns[index])[noval].sum().astype('Int32')
                if (morevals[i]>lessvals[i]):
                    
                    tree[node][dataset.groupby(dataset.columns[index]).label.count().index[i]] =  yesval 
                else:
                   
                    tree[node][dataset.groupby(dataset.columns[index]).label.count().index[i]] =  noval 
                
            else:    
                isolateddata = dataset.groupby(dataset.columns[index])
                isolateddata = isolateddata.get_group(dataset.groupby(dataset.columns[index]).label.count().index[i])
                isolateddata = isolateddata.drop(columns=dataset.columns[index])
                if ((len(isolateddata.columns)-1)==1):
                    morevals = isolateddata.groupby([isolateddata.columns[index],isolateddata.columns[-1]]).label.count().unstack().groupby(isolateddata.columns[index])[yesval].sum().astype('Int32')
                    lessvals = isolateddata.groupby([isolateddata.columns[index],isolateddata.columns[-1]]).label.count().unstack().groupby(isolateddata.columns[index])[noval].sum().astype('Int32')
                    if (morevals[0]>lessvals[0]):
                        
                        tree[node][dataset.groupby(dataset.columns[index]).label.count().index[i]] =  yesval 
                    else:
                    
                        tree[node][dataset.groupby(dataset.columns[index]).label.count().index[i]] =  noval 
                      
                else:
        
                    branch = id3(isolateddata, yesval, noval)
                    tree[node][dataset.groupby(dataset.columns[index]).label.count().index[i]] = branch
    return tree

By defining the output variables "yesval" and "noval" we build a decision tree using the train dataframe

In [6]:
yes = ">50K"
no = "<=50K"

tree = id3(traindata, yes, no)

In [7]:
tree

{'relationship': {' Husband': {'education': {' 10th': {'occupation': {' Adm-clerical': '<=50K',
      ' Craft-repair': {'workclass': {' Private': {'race': {' Asian-Pac-Islander': '<=50K',
          ' Black': {'marital-status': {' Married-civ-spouse': '<=50K'}},
          ' White': {'marital-status': {' Married-civ-spouse': '<=50K'}}}},
        ' Self-emp-not-inc': '<=50K'}},
      ' Exec-managerial': {'workclass': {' Local-gov': '<=50K',
        ' Private': '<=50K',
        ' Self-emp-not-inc': '<=50K',
        ' State-gov': '>50K'}},
      ' Farming-fishing': '<=50K',
      ' Handlers-cleaners': '<=50K',
      ' Machine-op-inspct': {'workclass': {' Private': {'marital-status': {' Married-civ-spouse': '<=50K'}},
        ' Self-emp-not-inc': '<=50K'}},
      ' Other-service': '<=50K',
      ' Protective-serv': '<=50K',
      ' Sales': {'workclass': {' Private': {'marital-status': {' Married-civ-spouse': '<=50K'}},
        ' Self-emp-inc': '<=50K',
        ' Self-emp-not-inc': '<=50K'}},